# (Draft) Storm Typing Notebook

The purpose of this notebook is to aid in the storm typing process by giving users easy access to select CONUS404 data, which dates back to late 1979. Users can select certain variables at a given interval, date, and duration, and create a plot for each one. After analyzing each plot, users score each plot on whether they contain synoptic, mesoscale convective, local convective, or tropical characteristics. At the end, a table is created and the scores are summed to give an overview on the possible storm type. 

[**CONUS404**: The NCAR–USGS 4-km Long-Term Regional Hydroclimate Reanalysis over the CONUS](https://journals.ametsoc.org/view/journals/bams/104/8/BAMS-D-21-0326.1.xml)

In [ ]:
from constants import *


from data_retrieval import *


from plot_functions import *

## Parameters

**Optional Parameters:**
 
 - `SBCAPE`: Surface Based Convective Available Potential Energy

 - `PWAT`: Precipitable Water

 - `PREC_ACC_NC`: Accumulated Precipitation (precip_accum_interval must be set to desired interval)

 - `PSFC`: Mean Sea Level Pressure

 - `SRH03`: 0-3km Storm Relative Helecity

 - `Z_50000Pa`: Geopotential Height at 50,000Pa(500mb)


In [ ]:
storm_params = {
    "vars_2d": {
        "SBCAPE": 6,
        "PREC_ACC_NC": 1,
        "PWAT": 6,
        "PSFC": 6,
        "SRH03": 6,
    },
    "precip_accum_interval": 6,
    "start_date_str": "2017/09/01-00z",
    "duration_hours": 24,
}

## Summary Statistics


Name                 | Description                         | Value
:--------------------|:------------------------------------|:------
mid_latitude_cyclone | Mid latitude cyclones are large scale events often characterized by surface low pressure systems and mid-upper level troughs. These events are frequently associated with higher wind shear and don't need to have much CAPE for higher precipitation amounts.   |yes = 1, no = 0
mesoscale_convective |Mesoscale convective systems are smaller in coverage and are often associated with high moisture and CAPE values. These events can have locally high wind shear but widespread high windshear is not common with these events.                                                            |yes = 1, no = 0
tropical             | Tropical events are associated with tropical(or post-tropical) cyclones mostly found during the months of June through November. These events are characterized by low pressure systems originating over the ocean and often contain extremely high moisture values. The Tropical Events section greatly aids in determining this storm type.                                                                 |yes = 1, no = 0
local_storm          | Local storms are very sparse in coverage, generally associated with low shear and higher CAPE values.                                                             |yes = 1, no = 0
Notes                | Additional comments                                                            |additional text


#### Provide some explanation of what is happening in the following cell

In [ ]:
vars_dates = process_date_intervals(
    storm_params["start_date_str"],
    storm_params["duration_hours"],
    storm_params["vars_2d"],
)

## Tropical Events

A check for tropical events that intersect the Trinity Basin during the time period between the start date and end duration. If an event is found, the name and start/end date of the storm will be listed.

In [ ]:
tropical_storms = search_tropical_events(
    storm_params["start_date_str"], storm_params["duration_hours"]
)
if tropical_storms:
    for storm_name, storm_info in tropical_storms.items():
        print(f"Storm name: {storm_name}")
        print(f"Start date for {storm_name}: {storm_info['start_date']}")
        print(f"End date for {storm_name}: {storm_info['end_date']}")
else:
    print("No Tropical Events Found")

## Accumulated Precipitation
Accumulated CONUS404 grid scale precipitation over a given interval, measured in inches.

In [ ]:
if "PREC_ACC_NC" in vars_dates:
    gif_plot = precip_plotter(
        vars_dates, storm_params, "PREC_ACC_NC", get_precip_dataset, plot_PREC_ACC_NC
    )
else:
    gif_plot = None
gif_plot

### Quantitative Assessment

In [ ]:
import pandas as pd
import scrapbook as sb

try:
    from notebook_utils import *
except:
    import sys

    sys.path.append(r"\Users\sjanke\Code\stormcloud\stormcloud\stormtyper")
    from notebook_utils import *


summary_stats = init_stats_dict()

In [ ]:
create_select_widget("PREC_ACC_NC", summary_stats)
create_notes_widget("PREC_ACC_NC", summary_stats)

# Surface Based CAPE

Convective Available Potential Energy (CAPE) is a measure of energy available for convection, measured in J/kg. Higher values indicate greater chances for thunderstorms and severe weather. Surface Based CAPE is CAPE calculated using a parcel based at the surface.

In [ ]:
if "SBCAPE" in vars_dates:
    gif_plot = alt_plotter(vars_dates, "SBCAPE", get_2d_dataset, plot_SBCAPE)
else:
    gif_plot = None
gif_plot

In [ ]:
create_select_widget("SBCAPE", summary_stats)
create_notes_widget("SBCAPE", summary_stats)

## Mean Sea Level Pressure

Mean Sea Level Pressure(MSLP) is the atmospheric pressure that would exist at sea level directly below a given surface pressure value, measured in hectopascals. MSLP is very useful for tracking surface high and low pressure systems as well as fronts. More on pressure definitions can be found [here.](https://www.weather.gov/bou/pressure_definitions)

In [ ]:
if "PSFC" in vars_dates:
    gif_plot = alt_plotter(vars_dates, "PSFC", get_2d_dataset, plot_SLP, alt_var="SLP")
else:
    gif_plot = None
gif_plot

In [ ]:
create_select_widget("SLP", summary_stats)
create_notes_widget("SLP", summary_stats)

## 0-3km Storm Relative Helicity

0-3km Storm Relative Helicity (SRH) is a measure of wind shear from the ground to roughly 10,000 ft. AGL, measured in m^2/s^2. Higher values indicate more wind shear, with values above 250 m^2/s^2 being generally considered high. High SRH values are often associated with frontal and synoptic systems, due to the increased pressure gradients and directional wind shifts those systems often create. 

In [ ]:
if "SRH03" in vars_dates:
    gif_plot = alt_plotter(vars_dates, "SRH03", get_2d_dataset, plot_SRH03)
else:
    gif_plot = None
gif_plot

In [ ]:
create_select_widget("SRH", summary_stats)
create_notes_widget("SRH", summary_stats)

## Precipitable Water

Precipitable water is a measure of moisture throughout the atmosphere, measured in inches. It's defined as the measure of water vapor in a vertical column of air if it were condensed and collected. The higher the value, the more moisture there is within the atmosphere, and the more likely heavy rain and flooding is to occur. 

In [ ]:
if "PWAT" in vars_dates:
    gif_plot = alt_plotter(vars_dates, "PWAT", get_2d_dataset, plot_PWAT)
else:
    gif_plot = None
gif_plot

In [ ]:
create_select_widget("PWAT", summary_stats)
create_notes_widget("PWAT", summary_stats)

## 500mb Geopotential Height

Geopotential height is the altitude of air in meters at a given pressure level, in this case 500mb. Geopotential height is very useful for locating mid-level troughs and ridges, which heavily influence synoptic systems at the surface. More about geopotential height can be found [here.](https://www.weather.gov/source/zhu/ZHU_Training_Page/Miscellaneous/Heights_Thicknesses/thickness_temperature.htm)

In [ ]:
if "Z_50000Pa" in vars_dates:
    gif_plot = alt_plotter(vars_dates, "Z_50000Pa", get_3d_dataset, plot_Z_50000Pa)
else:
    gif_plot = None
gif_plot

In [ ]:
create_select_widget("Z50000", summary_stats)
create_notes_widget("Z50000", summary_stats)

## Storm Assesment

In [ ]:
df = pd.DataFrame(summary_stats)
df

In [ ]:
storm_type_analysis(df)

In [ ]:
sb.glue("summary_stats", summary_stats)

# End